In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ciplab/real-and-fake-face-detection")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/real-and-fake-face-detection


In [28]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import glob

In [29]:
base_path = '/kaggle/input/real-and-fake-face-detection/real_and_fake_face'

In [30]:
real_path = glob.glob(os.path.join(base_path,'training_real','*'))
fake_path = glob.glob(os.path.join(base_path,'training_fake','*'))

all_paths = np.array(real_path+fake_path)

In [57]:
img_size = 240
batch_size = 32

In [32]:
all_labels = np.array([0]*len(real_path)+[1]*len(fake_path))

In [33]:
print(f'Number of real images in the dataset: {len(real_path)}')
print(f'Number of fake images in the dataset: {len(fake_path)}')
print(f'Total number of images in the dataset: {len(all_labels)}')

Number of real images in the dataset: 1081
Number of fake images in the dataset: 960
Total number of images in the dataset: 2041


In [34]:
from sklearn.model_selection import train_test_split

train_paths,val_paths,train_labels,val_labels = train_test_split(all_paths,all_labels,train_size=0.8,stratify=all_labels,random_state=6)

In [35]:
print(f'Train: {len(train_paths)} | Val: {len(val_paths)}')

Train: 1632 | Val: 409


In [36]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

In [58]:
def load_and_preprocess(path,label,augment=False):
  image_bytes = tf.io.read_file(path)
  img = tf.image.decode_jpeg(image_bytes,channels=3)
  img = tf.image.resize(img,[img_size,img_size])
  img = tf.keras.applications.efficientnet.preprocess_input(img)

  if augment:
    img = data_augmentation(img)

  return img,label

In [59]:
def build_dataset(paths,labels,shuffle_data=True,augment=False):
  data = tf.data.Dataset.from_tensor_slices((paths,labels))

  if shuffle_data:
    data = data.shuffle(buffer_size=1000)

  data = data.map(lambda x,y: load_and_preprocess(x,y,augment),num_parallel_calls=tf.data.AUTOTUNE)
  data = data.batch(32).prefetch(tf.data.AUTOTUNE)

  return data

In [60]:
train_paths = tf.convert_to_tensor(train_paths)
train_labels = tf.convert_to_tensor(train_labels)

val_paths = tf.convert_to_tensor(val_paths)
val_labels = tf.convert_to_tensor(val_labels)

In [61]:
train_ds = build_dataset(train_paths,train_labels,shuffle_data=True,augment=True)
val_ds = build_dataset(val_paths,val_labels,shuffle_data=False,augment=False)

In [62]:
from tensorflow.keras import layers,models

base_model = tf.keras.applications.EfficientNetB1(
    input_shape=(img_size,img_size,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True

for layer in base_model.layers[:100]:
  layer.trainable = False

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [77]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [78]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [79]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb1 (Functional)     │ (None, 8, 8, 1280)     │     6,575,239 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,747,528 (25.74 MB)

 Trainable params: 6,608,501 (25.21 MB)

 Non-trainable params: 139,027 (543.08 KB)

In [80]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [81]:
history = model.fit(train_ds,validation_data=val_ds,epochs=20,callbacks=[early_stop])

Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 125s 881ms/step - accuracy: 0.8357 - loss: 0.3612 - val_accuracy: 0.8166 - val_loss: 0.7545
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 33s 644ms/step - accuracy: 0.9121 - loss: 0.2672 - val_accuracy: 0.7751 - val_loss: 1.0436
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 42s 665ms/step - accuracy: 0.9174 - loss: 0.2018 - val_accuracy: 0.7531 - val_loss: 0.8517
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 41s 658ms/step - accuracy: 0.9360 - loss: 0.1670 - val_accuracy: 0.6846 - val_loss: 1.2235


In [ ]:
model.save("model.keras")